### CLEANSED - Criação da estrutura de banco de dados e tabelas no Hive


- Uma database deve ser criada para cada camada do datalake, com exceção da **raw**. Nessa camada ficarão os arquivos originais.
    - para se trabalhar com Raw data, deverá ser utilizada a camada **cleansed**
    - para o trabalho, não criaremos database para as camadas **sandbox** e **experiment**

#### 1) Acessando o bash do container hive-server

<img src="files/docker_hive-server.png" width="1000" height="500" align="left">


#### 2) Acessando o console beeline do hive

<img src="files/hive_beeline.png" width="600" height="300" align="left">


#### 3) Criando as bases de dados 

- criada uma base de dados para cada camada do datalake

````sql
CREATE DATABASE IF NOT EXISTS olist_cleansed_db WITH DBPROPERTIES('LOCATION'='/datalake/olist/cleansed');
CREATE DATABASE IF NOT EXISTS olist_enriched_db WITH DBPROPERTIES('LOCATION'='/datalake/olist/enriched');
CREATE DATABASE IF NOT EXISTS olist_consumption_db WITH DBPROPERTIES('LOCATION'='/datalake/olist/consumption');
CREATE DATABASE IF NOT EXISTS olist_trusted_db WITH DBPROPERTIES('LOCATION'='/datalake/olist/trusted');
````

<img src="files/hive_databases.png" width="1000" height="500" align="left">


#### 4) Criando tabela(s) 

##### 4.1) criação da tabela temporaria (sem partiçao) **geolocation_tb** para armazenar os dados dos correios extraidos a partir do CEP do dataset geolocation. 

````sql
USE olist_cleansed_db;

CREATE EXTERNAL TABLE olist_cleansed_db.geolocation_tb 
(
    geolocation_zip_code_prefix string,
    geolocation_city string,
    geolocation_state string,
    geolocation_street string
 )
 ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
 LINES TERMINATED BY '\n'
 tblproperties("skip.header.line.count"="1");

````

<img src="files/geolocation_temp.png" width="600" height="300" align="center">

##### 4.2) importando os dados da csv no datalake para a tabela temporária **geolocation_tb**

````sql
LOAD DATA INPATH '/datalake/olist/raw/csv/geolocation_cep_correios.csv'
OVERWRITE INTO TABLE geolocation_tb;
````

*ATENÇÃO: Não foi possível verificar se o código acima funcionou pois ao executar o select count para checagem, o container encerrou inesperadamente.* 

###### 4.3) criando tabela definitiva **geolocation_external_tb**, particionada por UF, para armazenar os dados atualizados dos CEP

````sql
CREATE EXTERNAL TABLE olist_cleansed_db.geolocation_external_tb 
(
    geolocation_zip_code_prefix string,
    geolocation_city string,
    geolocation_street string
 )
 PARTITION BY (geolocation_state string)
 ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
 LOCATION '/datalake/olist/cleansed/geolocation_external_tb';
````

<img src="files/geolocation_partition.png" width="600" height="300" align="center">


##### 4.4) carregando os dados da tabela temporária **geolocation_tb** para tabela definitiva **geolocation_external_tb**

````sql
INSERT OVERWRITE TABLE olist_cleansed_db.geolocation_external_tb PARTITION(geolocation_state) 
SELECT geolocation_zip_code_prefix, geolocation_city, geolocation_street, geolocation_state
FROM  geolocation_tb;
````

*ATENÇÃO: Ao executar o código acima o container é encerrado inesperadamente* 

##### 4.5) removendo a tabela temporaria **geolocation_tb**

````sql
ALTER TABLE olist_cleansed_db.geolocation_tb SET TBLPROPERTIES('EXTERNAL'='FALSE') ;
DROP TABLE olist_cleansed_db.geolocation_tb;
SHOW TABLES;
````

*ATENÇÃO: Esse código não foi executado por conta do problema encontrado no item 4.4*